#Import management

In [33]:
#we import sys to add the models folder to path, allowing us to use those scripts in visualization
import sys
sys.path.insert(1, '../models/')

import pandas as pd
import numpy as np
import panel as pn
import clean_data as cd

pn.extension('tabulator')

import hvplot.pandas

--Data Management--
Now, we get the .csv files with raw data from the '/data' folder.

In [34]:
data_path = '../data/all_sections.csv'

--File Reading--

We're now going to read the files that we got beforehand.

In [35]:
data = pd.read_csv(data_path)
data.columns

Index(['OrgDefinedId', 'Secciones',
       'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Semana 1\Week1 Subtotal Numerator',
       'Semana 1\Week1 Subtotal Denominator',
       'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\W

...And now we'll call the data cleaning scripts on this data.

In [36]:
#We rename the columns to make them easier to work with
data.rename(columns={
    #WEEK 1
    'OrgDefinedId':'user',
    'Secciones':'sec',
    'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1AM',
    'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1PM',
    'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2AM',
    'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2PM',
    'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3AM',
    'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3PM',
    'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4AM',
    'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4PM',

    'Semana 1\Week1 Subtotal Numerator':'week_1_numerator',
    'Semana 1\Week1 Subtotal Denominator':'week_1_denominator',
    
    #WEEK 2
    'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5PM',
    'Entrega de la mañana (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5AM',
    'Entrega de la mañana (D6) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'6AM',

    'Semana 2\Week 2 Subtotal Numerator':'week_2_numerator',
    'Semana 2\Week 2 Subtotal Denominator':'week_2_denominator',

    'Calculated Final Grade Numerator':'final_numerator',
    'Calculated Final Grade Denominator':'final_denominator'
}, inplace = True)

#drop unnecesary columns
data.drop(columns={'Adjusted Final Grade Numerator', 'Adjusted Final Grade Denominator','End-of-Line Indicator'}, axis=1, inplace=True)
#filter blank students
data = data[data['sec'].notna()]
#fill NaN with 0
data.fillna(0.0, inplace=True)

#change sections to numbers, this will be useful for binding on analytic tables later
data['sec'].replace(['Sección No. 1', 'Sección No. 2', 'Sección No. 3','Sección No. 4','Sección No. 5','Sección No. 6','Sección No. 7',
'Sección No. 8','Sección No. 9','Sección No. 10 - Las morcillitas'], [1,2,3,4,5,6,7,8,9,10])


,user,sec,1AM,1PM,2AM,2PM,3AM,3PM,4AM,4PM,week_1_numerator,week_1_denominator,5PM,5AM,6AM,week_2_numerator,week_2_denominator,final_numerator,final_denominator
1,#1000594810,Sección No. 3,0.0,0.0,3.5,5.0,2.5,0.0,2.5,0.0,13.5,20.0,0.0,5.0,0.0,5.0,5.0,18.5,25.0
2,#101051396,Sección No. 7,0.0,5.0,3.5,5.0,0.0,5.0,5.0,0.0,23.5,25.0,5.0,0.0,5.0,10.0,10.0,33.5,35.0
3,#1010840777,Sección No. 1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,35.0,35.0,0.0,0.0,0.0,0.0,0.0,35.0,35.0
4,#1010961769,Sección No. 4,5.0,5.0,5.0,5.0,3.5,5.0,5.0,0.0,33.5,35.0,5.0,5.0,5.0,20.0,20.0,53.5,55.0
5,#1010962618,Sección No. 5,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,35.0,35.0,5.0,5.0,5.0,15.0,15.0,50.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,#1141715139,Sección No. 2,5.0,5.0,5.0,5.0,5.0,3.5,5.0,5.0,38.5,40.0,3.5,3.5,5.0,12.0,15.0,50.5,55.0
113,#1145924881,Sección No. 1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,40.0,40.0,5.0,5.0,0.0,15.0,15.0,55.0,55.0
114,#202225959,Sección No. 10 - Las morcillitas,0.0,3.5,5.0,3.5,5.0,5.0,5.0,0.0,27.0,30.0,5.0,5.0,5.0,15.0,15.0,42.0,45.0
116,#665101,Sección No. 2,5.0,3.5,5.0,5.0,5.0,3.5,5.0,5.0,37.0,40.0,5.0,5.0,5.0,15.0,15.0,52.0,55.0
